In [ ]:
import torch
import cv2
import ipywidgets
from cnn import *
import numpy as np
cnn=Net()
# 加载网络模型
cnn.load_state_dict(torch.load('\params\\60-0.9709-net_params.pts',map_location='cpu'))

In [ ]:
# def findRoi(frame, thresValue):
#     rois = []
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray1 = cv2.dilate(gray,None,iterations=2)
#     gray2 = cv2.erode(gray1,None,iterations=2)
#     edges = cv2.absdiff(gray,gray2)
#     x = cv2.Sobel(edges,cv2.CV_16S,1,0)
#     y = cv2.Sobel(edges,cv2.CV_16S,0,1)
#     absX = cv2.convertScaleAbs(x)
#     absY = cv2.convertScaleAbs(y)
#     dst = cv2.addWeighted(absX,0.5,absY,0.5,0)
#     ret, ddst = cv2.threshold(dst,thresValue,255,cv2.THRESH_BINARY)
#     contours, hierarchy = cv2.findContours(ddst,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     area = []
#     rois = []
#     for k in range(len(contours)):
#         area.append(cv2.contourArea(contours[k]))
#     if area  :  
#         max_idx = np.argmax(np.array(area))
#         x, y, w, h = cv2.boundingRect(contours[max_idx])
#         rois.append((x,y,w,h))
#     return rois, edges,area
def findRoi(frame, thresValue,kernel):
    rois = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.dilate(gray,kernel,iterations=2)
    # gray2 = cv2.erode(gray1,None,iterations=2)
    ret, ddst = cv2.threshold(gray1,thresValue,255,cv2.THRESH_BINARY_INV)
    # contours, hierarchy = cv2.findContours(ddst,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours, hierarchy = cv2.findContours(ddst,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    area = []
    rois = []
    for k in range(len(contours)):
        area.append(cv2.contourArea(contours[k]))
    if area  :  
        max_idx = np.argmax(np.array(area))
        x, y, w, h = cv2.boundingRect(contours[max_idx])
        rois.append((x,y,w,h))
    return rois, ddst,area
    
    
    

def findDigit(cnn, roi, thresValue):
    ret, th = cv2.threshold(roi, thresValue, 255, cv2.THRESH_BINARY)
    th = cv2.resize(th,(28,28),interpolation=cv2.INTER_AREA)
    newing=[[th]]
    newing = torch.Tensor(np.array(newing)).float()/255. 
    predicions=cnn(newing)
    retult=torch.argmax(predicions).detach().numpy()
    return retult,th
def concatenate(images):
    n = len(images)
    output = np.zeros(20*20*n).reshape(-1,20)
    for i in range(n):
        output[20*i:20*(i+1),:] = images[i]
    return output

In [ ]:
font_scale=1.5 #字体大小
font=cv2.FONT_HERSHEY_PLAIN#字体类型
kernel = np.ones((4, 4), np.uint8)
wid = ipywidgets.Image()
cap=cv2.VideoCapture(0)
if not cap.isOpened():
    cap=cv2.VideoCapture(1)
if not cap.isOpened():
    raise IOError('Can not open video')
cap.set(cv2.CAP_PROP_FPS, 30)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,160)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,120)    
ret,img = cap.read()    
cntr=0;
display(wid)
while True:
    ret,frame=cap.read()
    cntr=cntr+1;
    #if((cntr%2)==0):
    if(1):
        rois,edges,area = findRoi(frame, 70,kernel)
        digits = []
        if rois: 
            x, y, w, h=rois[0]
            x_small=int(x+(w-h)/2)
            if  x_small<0:
                 x_small = 0
            # digit,th = findDigit(cnn,edges[y:y+h,x:x+w], 70)
            digit,th = findDigit(cnn,edges[y:y+h, x_small: x_small+h], 20)
            cv2.rectangle(frame, (int(x+(w-h)/2),y), (int(x+(w+h)/2),y+h), (153,153,0), 2)
            cv2.putText(frame, str(digit), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (127,0,255), 2)
        #cv2.putText(frame, str(digit), (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (127,0,255), 2)
        newEdges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
        newFrame = np.hstack((frame,newEdges))
        wid.value = cv2.imencode('.jpg',newFrame)[1].tobytes()

In [4]:
# -*- coding:utf-8 -*- 
import numpy as np
import imutils
import time
import cv2
# import V_Display as vd
# import V_UCom as com

# # Send_data
# Uart_buf = [0x55,0xAA,0x00,0x00,0x00,0x00,0x00,0x00,
#             0x00,0x00,0x00,0xAA]


if __name__ == '__main__':
    # Setup Usart
    # com.init(mode=2)

    # Setup SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 20
    params.maxThreshold = 100

    # Filter by Color.
    params.filterByColor = True
    params.blobColor = 0

    # Create a detector with the parameters
    # ver = (cv2.__version__).split('.')
    # if int(ver[0]) < 3 :
    #     detector = cv2.SimpleBlobDetector(params)
    # else :
    #     detector = cv2.SimpleBlobDetector_create(params)
    detector = cv2.SimpleBlobDetector_create(params)
    #增加了底层驱动，可以直接通过cv2的0号设备读取摄像头
    cap = cv2.VideoCapture(0)
    # cap.set(3,320)#设置摄像头输出宽
    # cap.set(4,240)#设置摄像头输出高
    print("start reading video...")
    time.sleep(2.0)
    print("start working")

    #初始化右侧图像显示功能,优化了传输,
    #增加了imshow('name',frame)函数，与cv2的imshow保持一致
    #无需指定fps
    # vd.init()

    while(True):
        ret, frame = cap.read()
        # frame = imutils.resize(frame, width=160)
        keypoints = detector.detect(frame)
        if(keypoints):
            size = 0#keypoints[0].size
            #for i in range (0, len(keypoints)):
            for a in keypoints:
#                x = keypoints[i].pt[0]
#                y = keypoints[i].pt[1]
                x = a.pt[0]
                y=a.pt[1]
                if(a.size > size):
                    Postion_x = int(x)
                    Postion_y = int(y)
        else:
            Postion_x = 80
            Postion_y = 60
        # Uart_buf = bytearray([0x55,0xAA,0x10,Postion_x>>8,Postion_x & 0x00ff,
        #                    Postion_y>>8,Postion_y & 0x00ff,0x00,0x00,0x00,0x00,0xAA])
        # com.send(Uart_buf)
        # im_with_keypoints = cv2.drawKeypoints(frame, keypoints, np.array([]), (255,255,255),cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)
        cv2.drawKeypoints(frame, keypoints, frame, (255,255,255),flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        # vd.show(im_with_keypoints)
        cv2.imshow('img',frame)
        print("\r{}".format(Postion_x,Postion_y), end="")
        if cv2.waitKey(1)==27:
            break
    cap.release()
    cv2.destroyAllWindows()


start reading video...
start working
809

error: OpenCV(4.5.5) ..\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 